<br>

<div align=center><font color=maroon size=6 style="line-height:40px;"><b>Masking and padding with Keras</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 Core: <a href="https://www.tensorflow.org/guide" style="text-decoration:none;">TensorFlow Guide</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Guide > <a href="https://www.tensorflow.org/guide/keras/masking_and_padding" style="text-decoration:none;">Masking and padding with Keras</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/snapshot-keras/site/en/guide/keras/masking_and_padding.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

In [5]:
print(tf.__version__)

2.8.0


<br>
<br>
<br>

## Introduction
<br>
<br>

<font style="color:maroon;font-size:115%"><font size=4 color=blue>**Masking**</font> is a way to tell sequence-processing layers that certain timesteps in an input are missing, and thus should be skipped when processing the data.

<br>
<br>
    
<font size=4 color=blue>**Padding**</font> is a special form of masking where the masked steps are at the start or the end of a sequence. Padding comes from the need to encode sequence data into contiguous batches: in order to make all sequences in a batch fit a given standard length, it is necessary to pad or truncate some sequences.</font>

<br>
<br>

Let's take a close look.

<br>
<br>
<br>

## Padding sequence data

<font style="color:maroon;font-size:115%">When processing sequence data, it is very common for individual samples to have different lengths.</font> Consider the following example (text tokenized as words):

```
[
  ["Hello", "world", "!"],
  ["How", "are", "you", "doing", "today"],
  ["The", "weather", "will", "be", "nice", "tomorrow"],
]
```

After vocabulary lookup, the data might be vectorized as integers, e.g.:

```
[
  [71, 1331, 4231]
  [73, 8, 3215, 55, 927],
  [83, 91, 1, 645, 1253, 927],
]
```

<font style="color:maroon;font-size:115%">The data is a nested list where individual samples have length 3, 5, and 6, respectively. Since the input data for a deep learning model must be a single tensor (of shape e.g. `(batch_size, 6, vocab_size)` in this case), samples that are shorter than the longest item need to be padded with some placeholder value (alternatively, one might also truncate long samples before padding short samples).

Keras provides a utility function to truncate and pad Python lists to a common length:
`tf.keras.preprocessing.sequence.pad_sequences`.</font>

In [6]:
raw_inputs = [[711, 632, 71],
              [73, 8, 3215, 55, 927],
              [83, 91, 1, 645, 1253, 927],
             ]

# By default, this will pad using 0s; it is configurable via the "value" parameter.
# Note that you could "pre" padding (at the beginning) or
# "post" padding (at the end).
# We recommend using "post" padding when working with RNN layers
# (in order to be able to use the CuDNN implementation of the layers).
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs, padding='post')

print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


<br>
<br>
<br>

## Masking

<font style="color:maroon;font-size:115%">Now that all samples have a uniform length, the model must be informed that some part of the data is actually padding and should be ignored. That mechanism is **masking**.
<br>
<br>

There are three ways to introduce input masks in Keras models:
<br>
<br>

- Add a `keras.layers.Masking` layer.
- Configure a `keras.layers.Embedding` layer with `mask_zero=True`.
- Pass a `mask` argument manually when calling layers that support this argument (e.g. RNN layers).</font>

<br>
<br>
<br>

## Mask-generating layers: `Embedding` and `Masking`

<font style="color:maroon;font-size:115%">Under the hood, these layers will create a mask tensor (2D tensor with shape `(batch, sequence_length)`), and attach it to the tensor output returned by the `Masking` or `Embedding` layer.</font>

In [7]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output)

tf.Tensor(
[[[ 0.04934898 -0.00727705  0.04687849  0.02233603  0.0474999
    0.02578664  0.00086989 -0.03996573 -0.00222905  0.03605564
    0.03060159  0.0157987   0.0052067   0.00235007 -0.00049876
   -0.04674941]
  [-0.04159208 -0.01111356  0.00392533  0.03859278 -0.00052519
   -0.03035796  0.02197576  0.0316926  -0.04414199 -0.01145703
   -0.02403643 -0.02590689 -0.00720479 -0.01609793 -0.01718944
   -0.03617406]
  [ 0.00433616  0.03567216  0.02141794 -0.0279493  -0.04524023
    0.00083884  0.00235515  0.00165167  0.03530927 -0.0388347
   -0.02749263  0.01181088 -0.01845618  0.02812867 -0.04005617
   -0.00173692]
  [ 0.04659328  0.02081953  0.03239347  0.04181651  0.00366927
   -0.02950674  0.00084295 -0.01304064  0.00669845  0.00811959
   -0.04965048 -0.00882796 -0.04379677  0.02121356 -0.04558191
   -0.01714968]
  [ 0.04659328  0.02081953  0.03239347  0.04181651  0.00366927
   -0.02950674  0.00084295 -0.01304064  0.00669845  0.00811959
   -0.04965048 -0.00882796 -0.04379677  0.021

<br>

In [10]:
print(masked_output._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


<br>

In [8]:
masking_layer = layers.Masking()
# Simulate the embedding lookup by expanding the 2D input to 3D, with embedding dimension of 10.

unmasked_embedding = tf.cast(tf.tile(tf.expand_dims(padded_inputs, axis=-1),
                                     [1, 1, 10]),
                             tf.float32)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding)

tf.Tensor(
[[[7.110e+02 7.110e+02 7.110e+02 7.110e+02 7.110e+02 7.110e+02 7.110e+02
   7.110e+02 7.110e+02 7.110e+02]
  [6.320e+02 6.320e+02 6.320e+02 6.320e+02 6.320e+02 6.320e+02 6.320e+02
   6.320e+02 6.320e+02 6.320e+02]
  [7.100e+01 7.100e+01 7.100e+01 7.100e+01 7.100e+01 7.100e+01 7.100e+01
   7.100e+01 7.100e+01 7.100e+01]
  [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
   0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
   0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
   0.000e+00 0.000e+00 0.000e+00]]

 [[7.300e+01 7.300e+01 7.300e+01 7.300e+01 7.300e+01 7.300e+01 7.300e+01
   7.300e+01 7.300e+01 7.300e+01]
  [8.000e+00 8.000e+00 8.000e+00 8.000e+00 8.000e+00 8.000e+00 8.000e+00
   8.000e+00 8.000e+00 8.000e+00]
  [3.215e+03 3.215e+03 3.215e+03 3.215e+03 3.215e+03 3.215e+03 3.215e+03
   3.215e+03 3.215e+03 3.215e+03]
  [5.500e+01 5.500e+01 5

<br>

In [9]:
print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


<br>

<font style="color:maroon;font-size:115%">As you can see from the printed result, the mask is a 2D boolean tensor with shape `(batch_size, sequence_length)`, **where each individual `False` entry indicates that the corresponding timestep should be ignored during processing**.</font>

<br>
<br>
<br>

## Mask propagation in the Functional API and Sequential API

<font style="color:maroon;font-size:115%">When using the **Functional API** or the **Sequential API**, a mask generated by an `Embedding` or `Masking` layer will be propagated through the network for any layer that is capable of using them (for example, RNN layers). Keras will automatically fetch the mask corresponding to an input and pass it to any layer that knows how to use it.</font>

For instance, in the following Sequential model, the `LSTM` layer will automatically receive a mask, which means it will ignore padded values:

In [11]:
model = keras.Sequential([layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True),
                          layers.LSTM(32),])

<br>

This is also the case for the following Functional API model:

In [12]:
inputs = keras.Input(shape=(None, ), dtype='int32')

x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

<br>
<br>
<br>

## Passing mask tensors directly to layers

<font style="color:maroon;font-size:115%">Layers that can handle masks (such as the `LSTM` layer) have a `mask` argument in their `__call__` method.

Meanwhile, layers that produce a mask (e.g. `Embedding`) expose a `compute_mask(input, previous_mask)` method which you can call.

Thus, you can pass the output of the `compute_mask()` method of a mask-producing layer to the `__call__` method of a mask-consuming layer, like this:</font>

In [14]:
class MyLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
        self.lstm = layers.LSTM(32)
    
    
    def call(self, inputs):
        x = self.embedding(inputs)
        
        # Note that you could also prepare a `mask` tensor manually.
        # It only needs to be a boolean tensor with the right shape, i.e. (batch_size, timesteps).
        mask = self.embedding.compute_mask(inputs)
        
        output = self.lstm(x, mask=mask)    # The layer will ignore the masked values
        
        return output


layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype("int32")
layer(x)

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[-0.00547996, -0.0054173 ,  0.00050939, ...,  0.00948612,
         0.00590012, -0.00375227],
       [ 0.00246026,  0.00042996, -0.00257897, ..., -0.00265325,
         0.00255879,  0.00134919],
       [ 0.0010486 , -0.00403494, -0.00099709, ..., -0.005806  ,
         0.00291907,  0.00738447],
       ...,
       [ 0.00238941,  0.00486492, -0.00209149, ...,  0.00288839,
         0.00978902,  0.00257493],
       [ 0.00204352, -0.00793534,  0.00110439, ..., -0.00617585,
         0.00553788,  0.00692509],
       [ 0.00603078, -0.00147527,  0.00333826, ..., -0.00777231,
         0.00315196, -0.00450323]], dtype=float32)>

<br>
<br>
<br>

## Supporting masking in your custom layers

<font style="color:maroon;font-size:115%">Sometimes, you may need to write layers that generate a mask (like `Embedding`), or layers that need to modify the current mask.</font>

For instance, any layer that produces a tensor with a different time dimension than its input, such as a `Concatenate` layer that concatenates on the time dimension, will need to modify the current mask so that downstream layers will be able to properly take masked timesteps into account.

<font style="color:maroon;font-size:115%">To do this, your layer should implement the `layer.compute_mask()` method, which produces a new mask given the input and the current mask.</font>

Here is an example of a `TemporalSplit` layer that needs to modify the current mask.

In [15]:
class TemporalSplit(keras.layers.Layer):
    """Split the input tensor into 2 tensors along the time dimension."""
    
    def call(self, inputs):
        # Expect the input to be 3D and mask to be 2D, split the input tensor into 2
        # subtensors along the time axis (axis 1).
        return tf.split(inputs, 2, axis=1)
    
    
    def compute_mask(self, inputs, mask=None):
        # Also split the mask into 2 if it presents.
        if mask is None:
            return None
        return tf.split(mask, 2, axis=1)


first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

tf.Tensor(
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]], shape=(3, 3), dtype=bool)
tf.Tensor(
[[False False False]
 [ True  True False]
 [ True  True  True]], shape=(3, 3), dtype=bool)


<br>

Here is another example of a `CustomEmbedding` layer that is capable of generating a mask from input values:

In [17]:
class CustomEmbedding(layers.Layer):
    def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
        super(CustomEmbedding, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.mask_zero = mask_zero
    
    
    def build(self, input_shape):
        self.embeddings = self.add_weight(shape=(self.input_dim, self.output_dim),
                                          initializer='random_normal',
                                          dtype='float32',)
    
    
    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)
    
    
    def compute_mask(self, inputs, mask=None):
        if not self.mask_zero:
            return None
        return tf.not_equal(inputs, 0)   # inputs 中元素 不等于0 的就为 True



layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype('int32')

y = layer(x)
mask = layer.compute_mask(x)

print(x)
print('\n\n')
print(mask)

[[4 3 1 8 7 4 8 8 6 3]
 [2 3 6 1 0 4 4 3 2 2]
 [4 8 5 0 3 3 7 0 1 1]]



tf.Tensor(
[[ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True False  True  True  True  True  True]
 [ True  True  True False  True  True  True False  True  True]], shape=(3, 10), dtype=bool)


<br>
<br>
<br>

## Opting-in to mask propagation on compatible layers

Most layers don't modify the time dimension, so don't need to modify the current mask.
However, they may still want to be able to **propagate** the current mask, unchanged, to the next layer. <font style="color:maroon;font-size:115%">**This is an opt-in behavior.** By default, a custom layer will destroy the current mask (since the framework has no way to tell whether propagating the mask is safe to do).

If you have a custom layer that does not modify the time dimension, and if you want it to be able to propagate the current input mask, you should set `self.supports_masking = True` in the layer constructor. In this case, the default behavior of `compute_mask()` is to just pass the current mask through.</font>

Here's an example of a layer that is whitelisted for mask propagation:

In [19]:
class MyActivation(layers.Layer):
    def __init__(self, **kwargs):
        super(MyActivation, self).__init__(**kwargs)
        
        # Signal that the layer is safe for mask propagation
        self.supports_masking = True
    
    
    def call(self, inputs):
        return tf.nn.relu(inputs)

<br>

<font style="color:maroon;font-size:115%">You can now use this custom layer in-between a **mask-generating layer** (like `Embedding`) and a **mask-consuming layer** (like `LSTM`), and it will pass the mask along so that it reaches the mask-consuming layer.</font>

In [22]:
inputs = keras.Input(shape=(None, ), dtype='int32')
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
x = MyActivation()(x)             # Will pass the mask along
print("Mask found:", x._keras_mask)

outputs = layers.LSTM(32)(x)    # Will receive the mask

model = keras.Model(inputs, outputs)

Mask found: KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.bool, name=None), name='Placeholder_1:0')


<br>
<br>
<br>

## Writing layers that need mask information

<font style="color:maroon;font-size:115%">Some layers are mask *consumers*: they accept a `mask` argument in `call` and use it to determine whether to skip certain time steps.

To write such a layer, you can simply add a `mask=None` argument in your `call` signature. The mask associated with the inputs will be passed to your layer whenever it is available.</font>

Here's a simple example below: a layer that computes a softmax over the time dimension (axis 1) of an input sequence, while discarding masked timesteps.

In [24]:
class TemporalSoftmax(layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_float_mask = tf.expand_dims(tf.cast(mask, 'float32'), -1)
        inputs_exp = tf.exp(inputs) * broadcast_float_mask
        inputs_sum = tf.reduce_sum(inputs_exp * broadcast_float_mask,
                                   axis=-1,
                                   keepdims=True)
        
        return inputs_exp / inputs_sum



inputs = keras.Input(shape=(None, ), dtype='int32')
x = layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
x = layers.Dense(1)(x)
outputs = TemporalSoftmax()(x)

model = keras.Model(inputs, outputs)
y = model(np.random.randint(0, 10, size=(32, 100)),
          np.random.random((32, 100, 1)))

<br>
<br>
<br>

## Summary

<font style="color:maroon;font-size:115%">That is all you need to know about padding & masking in Keras. To recap:

- "Masking" is how layers are able to know when to skip / ignore certain timesteps in sequence inputs.
<br>
    
- Some layers are mask-generators: `Embedding` can generate a mask from input values (if `mask_zero=True`), and so can the `Masking` layer.
<br>
    
- Some layers are mask-consumers: they expose a `mask` argument in their `__call__` method. This is the case for RNN layers.
<br>
    
- In the Functional API and Sequential API, mask information is propagated automatically.
<br>
    
- When using layers in a standalone way, you can pass the `mask` arguments to layers manually.
<br>
    
- You can easily write layers that modify the current mask, that generate a new mask, or that consume the mask associated with the inputs.</font>

<br>
<br>
<br>